# Neural Nets Project 3
### Jeremiah Pratt
#### Guide used for learning high-level Tensorflow: https://www.tensorflow.org/tutorials/estimators/cnn#building_the_cnn_mnist_classifier

In [23]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [10]:
f = gzip.open('mnist.pkl.gz','rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

In [21]:

train_feats.shape

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


(55000, 784)

In [24]:
TRAIN_SIZE = 50000
VAL_SIZE = 5000

def Data(object):
    def __init__(self):
        mnist = tf.contrib.learn.datasets.load_dataset("mnist")
        t_feats = mnist.train.images
        t_labels = np.asarray(mnist.train.labels,dtype=np.int32)
        test_feats = mnist.test.images
        test_labels = np.asarray(mnist.test.labels,dtype=np.int32)
        
        train_feats, val_feats, train_labels, val_labels = train_test_split(
            t_feats,t_labels, test_size=.09, random_state=123)
        

In [2]:
def cnn(features,labels,mode):
    
    #reshapes MNIST features from [1,784] to proper shape,
    #allowing for varying batch sizes as well
    inLayer = tf.reshape(features['x'],[-1,28,28,1])
    
    #define a l2 regularizer, for use on the convolutional layers
    l2reg = tf.contrib.layers.l2_regularizer(scale=0.1)
    
    #performs 5x5 convolutions, increases data space by
    #factor of 32
    conv1 = tf.layers.conv2d(
        inputs=inLayer, filters=32,kernel_size=[5,5],
        padding="same",activation=tf.nn.elu,
        kernel_regularizer = l2reg)
    
    #reduces size of data space by half through max
    #pooling
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, pool_size=[2,2], strides=2)
    
    #doubles size of data space again
    conv2 = tf.layers.conv2d(
        inputs=pool1, filters=64, kernel_size=[5,5],
        padding="same",activation=tf.nn.elu,
        kernel_regularizer = l2reg)
    
    #pooling again
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, pool_size=[2,2], strides=2)
    
    #reshape for the dense layer
    flat = tf.reshape(pool2, [-1, 7*7*64])
    
    #dense layer, duh
    dense = tf.layers.dense(
        inputs=flat, units=1024, activation=tf.nn.elu)
    
    #dropout regularization
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #classification layer
    classif = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=classif, axis=1),
        "probabilities": tf.nn.softmax(classif, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=.001)
        train_op = optimizer.minimize(
            loss = loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    #EVAL mode stuff
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels,predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode,loss=loss, eval_metric_ops=eval_metric_ops)

In [15]:
data = Data()

classif = tf.estimator.Estimator(model_fn=cnn)

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": data.train_feats},
    y=data.train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook]

eval_input_fn)

(50000, 784)
(50000,)


784